In [1]:
import math
import numpy as np
import pandas as pd
import os
import tempfile

from scipy.spatial.distance import pdist, squareform
import scipy.io as sio

import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
trainPredict = np.load('trainPredict.npy')
train_result = np.sum(trainPredict, axis=1)
testPredict = np.load('testPredict.npy')
test_result = np.sum(testPredict, axis=1)

##  Get distributions of Dest by Source,Time

In [3]:
orderdf = pd.read_csv('ignored_assets/orders.csv', parse_dates = [6])

In [4]:
# Create map of driver IDs
driver_ids=orderdf['driver_id'].unique()

# create map of core stations
core_stations = orderdf['start_district_hash'].unique()
core_stations.sort()

#Index stuff
orderdf_f=orderdf[(~pd.isnull(orderdf['driver_id'])) & orderdf['dest_district_hash'].isin(core_stations)]
myindex=pd.MultiIndex.from_arrays([orderdf_f['driver_id'],orderdf_f['timestamp']],names=('driver_id','timestamp'))

orderdf2=orderdf_f.set_index(myindex)
orderdf_s=orderdf2.sort_index()

orderdf_s['time_bucket'] = orderdf_s['timestamp'].dt.round('5min')
orderdf_s['pax_arrival_time'] = orderdf_s['timestamp'] + orderdf_s['expected_travel_time'] * pd.Timedelta('1 minute')
orderdf_s['time_bucket_arrival'] = orderdf_s['pax_arrival_time'].dt.round('5min')

In [19]:
stepsize = pd.Timedelta('5 min')
horizon = pd.Timedelta('2 hour')
start = pd.to_datetime('2016-01-21 00:00:00')
end = pd.to_datetime('2016-01-22 00:00:00')

In [20]:
mask = (orderdf_s['time_bucket'] >= start) & (orderdf_s['time_bucket'] < end)

In [21]:
pt = pd.pivot_table(orderdf_s[mask], index=['time_bucket','start_district_hash'], 
               columns=['dest_district_hash'], aggfunc=len, 
               values='order_id', fill_value=0)

In [24]:
pt

dest_district_hash                                    08232402614a9b48895cc3d0aeb0e9f2  \
time_bucket         start_district_hash                                                  
2016-01-21 00:00:00 08f5b445ec6b29deba62e6fd8b0325a6                                 0   
                    1afd7afbc81ecc1b13886a569d869e8a                                 0   
                    1c60154546102e6525f68cb4f31e0657                                 0   
                    1cbfbdd079ef93e74405c53fcfff8567                                 0   
                    1ecbb52d73c522f184a6fc53128b1ea1                                 0   
                    2301bc920194c95cf0c7486e5675243c                                 0   
                    2407d482f0ffa22a947068f2551fe62c                                 0   
                    2920ece99323b4c111d6f9affc7ea034                                 0   
                    364bf755f9b270f0f9141d1a61de43ee                                 0   
                    38d5ad2d22b61109fd8e7b43cd0e8901                                 0   
                    3a43dcdff3c0b66b1acb1644ff055f9d                                 0   
                    44c097b7bd219d104050abbafe51bd49                                 0   
                    4725c39a5e5f4c188d382da3910b3f3f                                 0   
                    4f8d81b5c31af5d1ba579a65ddc8a5cb                                 0   
                    52a4e8aaa12f70020e889aed8fd5ddbc                                 0   
                    52d7b69796362a8ed1691a6cc02ddde4                                 0   
                    52e56004d92b8c74d53e1e42699cba6f                                 0   
                    62afaf3288e236b389af9cfdc5206415                                 0   
                    693a21b16653871bbd455403da5412b4                                 0   
                    74c1c25f4b283fa74a5514307b0d0278                                 0   
                    82cc4851f9e4faa4e54309f8bb73fd7c                                 0   
                    87285a66236346350541b8815c5fae94                                 0   
                    90c5a34f06ac86aee0fd70e2adce7d8a                                 0   
                    91690261186ae5bee8f83808ea1e4a01                                 0   
                    929ec6c160e6f52c20a4217c7978f681                                 0   
                    a5609739c6b5c2719a3752327c5e33a7                                 0   
                    b05379ac3f9b7d99370d443cfd5dcc28                                 0   
                    b26a240205c852804ff8758628c0a86a                                 0   
                    b702e920dcd2765e624dc1ce3a770512                                 0   
                    ba32abfc048219e933bee869741da911                                 0   
...                                                                                ...   
2016-01-21 23:55:00 4b7f6f4e2bf237b6cc58f57142bea5c0                                 0   
                    4f8d81b5c31af5d1ba579a65ddc8a5cb                                 0   
                    52a4e8aaa12f70020e889aed8fd5ddbc                                 0   
                    52d7b69796362a8ed1691a6cc02ddde4                                 0   
                    52e56004d92b8c74d53e1e42699cba6f                                 0   
                    62afaf3288e236b389af9cfdc5206415                                 0   
                    693a21b16653871bbd455403da5412b4                                 0   
                    73ff8ef735e1d68f0cdcbb84d788f2b6                                 0   
                    74c1c25f4b283fa74a5514307b0d0278                                 0   
                    825c426141df01d38c1b9e9c5330bdac                                 0   
                    82cc4851f9e4faa4e54309f8bb73fd7c                                 0   
                    87285a66236346350541

In [23]:
66 * 288

19008